# MVD 13. cvičení
V dnešním cvičení se bude implementovat apriori a FP-Growth algoritmus pro nalezení častých vzorů.

### Úkol 1: Generování náhodných transakcí
1. Vytvořte Pandas DataFrame obsahující náhodné transakce.
    - Položky např.: ['mléko', 'chléb', 'máslo', 'vejce', 'sýr', 'pivo', 'víno', 'chipsy', 'ovoce', 'zelenina', ...]
    - Počet transakcí: Náhodně zvolte číslo mezi 50 až 100.
    - Počet položek v transakci: Každá transakce by měla obsahovat 2 až 6 náhodných položek.

2. Zobrazte prvních 5 transakcí DataFrame.

In [34]:
import pandas as pd
import numpy as np

In [35]:
def generate_df(trans_min: int = 50, trans_max: int = 101, trans_len_min: int = 2, trans_len_max: int = 6) -> pd.DataFrame:
    items = ['mléko', 'chléb', 'máslo', 'vejce', 'sýr', 'pivo', 'víno', 'chipsy', 'ovoce', 'zelenina',]
    transactions_number = np.random.randint(trans_min, trans_max)  # generate random num from the interval
    items_per_transaction = np.random.random_integers(trans_len_min, trans_len_max, transactions_number)  # generate `transactions_number` times random int from interval

    # lambda function to randolmly choose `n` elements from list
    rand_get = lambda item_list, item_num: np.random.choice(item_list, size=item_num, replace=False)
    data: list[np.array,] = [rand_get(items, item_num) for item_num in items_per_transaction]

    # create df
    df = pd.DataFrame({'transaction': data})
    df.index.name = 'id_trans'
    return df

In [36]:
df = generate_df()
df.head()

/var/folders/51/0prm0g2x0y13rpfcls9fpv180000gn/T/ipykernel_82294/240828542.py:4: DeprecationWarning: This function is deprecated. Please call randint(2, 6 + 1) instead
  items_per_transaction = np.random.random_integers(trans_len_min, trans_len_max, transactions_number)  # generate `transactions_number` times random int from interval


,transaction
id_trans,
0,"[víno, máslo, mléko, ovoce, zelenina, pivo]"
1,"[vejce, zelenina]"
2,"[chipsy, ovoce]"
3,"[chipsy, sýr, ovoce, víno, chléb]"
4,"[víno, zelenina, chléb]"


### Úkol 2: Implementace Apriori algoritmu
1. Napište funkci `apriori`, která:
    - Přijme DataFrame obsahující transakce.
    - Najde časté vzory (itemsety) na základě minimálního supportu (např. minsup = 0.5).
    - Vrátí seznam častých vzorů a jejich support hodnoty.

2. Otestujte funkci na vygenerovaných transakcích.

In [37]:
from itertools import combinations as itercomb

In [38]:
def apriori(df: pd.DataFrame, minsup: int = 2) -> list[str]:
    """
    Apriori algorithm.
    Find frequent itemsets (sets of items that appear together frequently in a dataset.

    Args:
        df: pandas.DataFrame with column `transaction` that contains a list of items (strings).
        minsup: - number of `support` of itemset to be considered as "frequent".

    Returns:
        all_itemsets: - list of itemsets that are considered as "frequent".
    """
    itemset_len = 1
    transactions_number = df.transaction.size
    no_more_candidates = False
    all_itemsets = []

    candidates_sets = []
    candidates_counts = []

    while not no_more_candidates:
        for transaction in df.transaction:
            # get all item combinations from transaction with `itemset_len` length of itemset
            combinations = itercomb(transaction, itemset_len)

            # per itemset:
            for itemset in combinations:
                sorted_itemset = sorted(itemset)

                # if `sorted_itemset` already exists, increase count by 1
                if sorted_itemset in candidates_sets:
                    idx = candidates_sets.index(sorted_itemset)
                    candidates_counts[idx] += 1
                # else add it and set `count` to 1
                else:
                    candidates_sets.append(sorted_itemset)
                    candidates_counts.append(1)

        # if there is no candidates with current `itemset_len` => return current global list of itemsets (exit)
        if len(candidates_sets) == 0:
            no_more_candidates = True

        # select only itemsets with `support` value >= minimum support value
        for idx, count in enumerate(candidates_counts):
            support = count / transactions_number
            if support >= minsup:
                all_itemsets.append((round(support, 2), candidates_sets[idx]))
        
        # empty all structures, increase itemset_len
        itemset_len += 1
        candidates_sets = []
        candidates_counts = []

    result_df = pd.DataFrame(all_itemsets)
    result_df.columns = ['support', 'itemsets']

    return result_df


In [39]:
minsup = 0.2
result = apriori(df, minsup)

print(f"RESULTS for minsup={minsup}:\n", '-' * 20)
print(result)

RESULTS for minsup=0.2:
 --------------------
    support            itemsets
0      0.35              [víno]
1      0.39             [máslo]
2      0.40             [mléko]
3      0.45             [ovoce]
4      0.44          [zelenina]
5      0.47              [pivo]
6      0.35             [vejce]
7      0.50            [chipsy]
8      0.32               [sýr]
9      0.39             [chléb]
10     0.21       [ovoce, víno]
11     0.24       [mléko, pivo]
12     0.23       [ovoce, pivo]
13     0.21    [pivo, zelenina]
14     0.21     [chipsy, ovoce]
15     0.24  [chipsy, zelenina]
16     0.24     [chipsy, mléko]
17     0.24      [chipsy, pivo]


### Úkol 3: Implementace FP-Growth algoritmu
 1. Použijte knihovnu mlxtend k použití FP-Growth algoritmu.
 2. Převádějte DataFrame na vhodný formát pomocí TransactionEncoder.
 3. Najděte časté vzory s minimálním supportem (např. minsup = 0.5).

In [40]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

In [41]:
def fp_growth(df: pd.DataFrame, minsup: float = 0.5):
    """
    """
    result = fpgrowth(df, min_support=minsup, use_colnames=True)
    return result


In [42]:
def tranform_df(df: pd.DataFrame) -> pd.DataFrame:
    # change dataframe structure to fit the algorithm
    te = TransactionEncoder()
    dataset = df.transaction.values
    te_ary = te.fit(dataset).transform(dataset)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

In [43]:
te_df = tranform_df(df)
te_df.head()

,chipsy,chléb,mléko,máslo,ovoce,pivo,sýr,vejce,víno,zelenina
0,False,False,True,True,True,True,False,False,True,True
1,False,False,False,False,False,False,False,True,False,True
2,True,False,False,False,True,False,False,False,False,False
3,True,True,False,False,True,False,True,False,True,False
4,False,True,False,False,False,False,False,False,True,True


In [44]:
result_df = fp_growth(te_df, 0.2)
result_df['support'] = result_df['support'].round(2)
result_df

,support,itemsets
0,0.47,(pivo)
1,0.45,(ovoce)
2,0.44,(zelenina)
3,0.40,(mléko)
4,0.39,(máslo)
5,0.35,(víno)
6,0.35,(vejce)
7,0.50,(chipsy)
8,0.39,(chléb)
9,0.32,(sýr)


### Úkol 4: Porovnání Apriori a FP-Growth
1. Porovnejte výsledky obou algoritmů:
    - Počet nalezených vzorů.
    - Výpočetní čas (měřte pomocí time, zkuste i zvýšit počet transakcí).

In [45]:
import time

In [58]:
# generate dfs
df_apriori = generate_df(trans_min=10000, trans_max=20000, trans_len_min=2, trans_len_max=10)
df_fpgrowth = tranform_df(df_apriori)

/var/folders/51/0prm0g2x0y13rpfcls9fpv180000gn/T/ipykernel_82294/240828542.py:4: DeprecationWarning: This function is deprecated. Please call randint(2, 10 + 1) instead
  items_per_transaction = np.random.random_integers(trans_len_min, trans_len_max, transactions_number)  # generate `transactions_number` times random int from interval


In [59]:
min_sup = 0.4

In [60]:
# test Apriori algorithm
start_time_apriori = time.time()
result_apriori = apriori(df_apriori, minsup=min_sup)
time_apriori = time.time() - start_time_apriori

In [61]:
# test FPGrowth algorithm
start_time_fpgrowth = time.time()
result_fpgrowth = fp_growth(df_fpgrowth, minsup=min_sup)
time_fpgrowth = time.time() - start_time_fpgrowth

In [62]:
# Compare results of two algorithms
print(f"---FPGrowth---\nresults number: {len(result_fpgrowth.itemsets)}")
print(f"time: {time_fpgrowth:.4f} sec")

print(f"---Apriori---\nresults number: {len(result_apriori.itemsets)}")
print(f"time: {time_apriori:.4f} sec")

---FPGrowth---
results number: 55
time: 0.9619 sec
---Apriori---
results number: 55
time: 15.9225 sec


In [63]:
result_apriori.head()

,support,itemsets
0,0.60,[pivo]
1,0.60,[chléb]
2,0.60,[zelenina]
3,0.61,[vejce]
4,0.60,[víno]


In [64]:
result_fpgrowth.head()

,support,itemsets
0,0.608821,(chipsy)
1,0.608629,(máslo)
2,0.605465,(vejce)
3,0.602780,(zelenina)
4,0.602780,(pivo)
